Uso de Gradio para la interfaz de prueba de un modelo de clasificación con algunas modificaciones en los componentes.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import gradio as gr
import io
import matplotlib.pyplot as plt
from PIL import Image
import io

# Lectura de datos de diabetes
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columnas = [
    "Embarazos",
    "Glucosa",
    "Presion_sanguinea",
    "Pliegue_cutaneo",
    "Insulina",
    "IMC",
    "DPF",
    "Edad",
    "Resultado",
]
datos = pd.read_csv(url, names=columnas)

# # Separación de las características y la variable objetivo
# X = datos.drop("Resultado", axis=1)
# y = datos["Resultado"]

# Entrenando el modelo solo con algunas características
X = datos[["Embarazos", "Glucosa", "Presion_sanguinea", "IMC", "Edad"]]
y = datos["Resultado"]

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Entrenando el modelo de regresión logística
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)


# Función para hacer predicciones
def predecir_diabetes(embarazos, glucosa, presion, imc, edad):
    entrada = pd.DataFrame(
        [[embarazos, glucosa, presion, imc, edad]],
        columns=["Embarazos", "Glucosa", "Presion_sanguinea", "IMC", "Edad"],
    )
    prediccion = modelo.predict(entrada)
    probabilidad = modelo.predict_proba(entrada)[0][1]
    resultado = "Positivo" if prediccion[0] else "Negativo"

    # Gráfico de probabilidad
    fig, ax = plt.subplots()
    categories = ["No Diabetes", "Diabetes"]
    valores_prob = [1 - probabilidad, probabilidad]
    ax.bar(categories, valores_prob, color=["blue", "red"])
    ax.set_ylim(0, 1)
    ax.set_ylabel("Probabilidad")
    ax.set_title("Distribución de probabilidades")

    # Guardando la figura en un buffer en memoria y convirtiéndola en imagen
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    imagen_resultado = Image.open(buf)

    return resultado, round(probabilidad * 100, 2), imagen_resultado


# Creación de la interfaz de Gradio con validaciones y descripciones
inputs = [
    gr.Number(
        label="Embarazos",
        minimum=0,
        maximum=5,
        step=1,
        precision=0,
        info="Número de embarazos [0-5]",
    ),
    gr.Number(
        label="Glucosa",
        minimum=0,
        maximum=199,
        step=1,
        precision=0,
        info="Nivel de glucosa en sangre [0-199]",
    ),
    gr.Number(
        label="Presión sanguínea",
        minimum=0,
        maximum=122,
        step=1,
        precision=0,
        info="Presión sanguínea (mm Hg) [0-122]",
    ),
    gr.Number(
        label="IMC",
        minimum=0,
        maximum=67.1,
        step=0.1,
        info="Índice de masa corporal [0-67.1]",
    ),
    gr.Number(
        label="Edad",
        minimum=1,
        maximum=81,
        step=1,
        precision=0,
        info="Edad en años [1-81]",
    ),
]

outputs = [
    gr.Text(label="Resultado de predicción"),
    gr.Text(label="Probabilidad (%)"),
    gr.Image(type="pil", label="Gráfico de probabilidad"),
]

iface = gr.Interface(
    fn=predecir_diabetes,
    inputs=inputs,
    outputs=outputs,
    title="Detección de diabetes",
)

# Desplegando la interfaz
iface.launch()

In [ ]:
# Deteniendo la demostración
iface.close()